# 图像卷积

## 互相关运算

In [1]:
import torch
from torch import nn
from d2l import torch as d2l

In [7]:
def corr2d(X, K):
    """计算二维互相关运算"""
    h, w = K.shape 
    # 定义输出层的大小
    Y = torch.zeros((X.shape[0]-h+1), (X.shape[1]-w+1))
    # 循环遍历求值
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            # 矩阵 X 的子矩阵和 核矩阵 K 相乘，再求和得到输出
            Y[i][j] = (X[i:i+h, j:j+w]*K).sum()
    return Y


In [9]:
X = torch.tensor([[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]])
K = torch.tensor([[0.0, 1.0], [2.0, 3.0]])
corr2d(X, K)

tensor([[19., 25.],
        [37., 43.]])

In [12]:
X = torch.rand(3,3)
K = torch.rand(2,2)
print(corr2d(X, K))
corr2d(X, K)+1

tensor([[1.2808, 0.9933],
        [0.9039, 1.0606]])


tensor([[2.2808, 1.9933],
        [1.9039, 2.0606]])

## 实现二维卷积层

In [13]:
class Conv2D(nn.Module):
    def __init__(self, kernel_size):
        super.__init__()
        self.weight = nn.Parameter(torch.rand(kernel_size))
        # 偏移量是个常数
        self.bias = nn.Parameter(torch.zeros(1))
    
    def forward(self, x):
        return corr2d(x, self.weight)+self.bias


卷积层的简单应用：检测图像中不同颜色的边缘

In [33]:
# X 是个图，改图的左右两侧各有一条边缘线，目标是检测图中的边缘
X = torch.ones((6,8))
X[:, 2:6] = 0
X

tensor([[1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.]])

In [26]:
# 如果元素没有改变，则输出 0，如果小块内的元素出现异号，那么输出 1或 -1
# 左起 从 1 到 0 输出 1；从 0 到 1 输出 -1
K = torch.tensor([[1.0,-1.0]])
print(K)
K.shape

tensor([[ 1., -1.]])


torch.Size([1, 2])

In [34]:
Y = corr2d(X, K)
print(Y)
Y.shape

tensor([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.]])


torch.Size([6, 7])

上述核只能检测垂直边缘，不能检测横向边

In [35]:
Y_T = corr2d(X.t(), K)
print(Y_T)
Y_T.shape

tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])


torch.Size([8, 5])

## 学习由 X 生成 Y 的卷积核

In [36]:
conv2d = nn.Conv2d(1, 1, kernel_size=(1,2), bias=False)

X = X.reshape((1,1,6,8))
Y = Y.reshape((1,1,6,7))
Y

tensor([[[[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
          [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
          [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
          [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
          [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
          [ 0.,  1.,  0.,  0.,  0., -1.,  0.]]]])

In [37]:
for i in range(10):
    Y_hat = conv2d(X)
    l = (Y_hat - Y)**2
    conv2d.zero_grad()
    l.sum().backward()
    conv2d.weight.data[:] -= 3e-2 * conv2d.weight.grad
    if (i+1)%2 == 0:
        print(f'bath {i+1}, loss {l.sum():.3f}')


bath 2, loss 6.353
bath 4, loss 1.818
bath 6, loss 0.613
bath 8, loss 0.229
bath 10, loss 0.090
